In [ ]:
import scipy.io as spio

def loadmat(filename):
    '''
    this function should be called instead of direct spio.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    '''
    data = spio.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_keys(data)

def _check_keys(dict):
    '''
    checks if entries in dictionary are mat-objects. If yes
    todict is called to change them to nested dictionaries
    '''
    for key in dict:
        if isinstance(dict[key], spio.matlab.mio5_params.mat_struct):
            dict[key] = _todict(dict[key])
    return dict        

def _todict(matobj):
    '''
    A recursive function which constructs from matobjects nested dictionaries
    '''
    dict = {}
    for strg in matobj._fieldnames:
        elem = matobj.__dict__[strg]
        if isinstance(elem, spio.matlab.mio5_params.mat_struct):
            dict[strg] = _todict(elem)
        else:
            dict[strg] = elem
    return dict


In [ ]:
for plotting
performance_fig, ax = plt.subplots(figsize = (12,8))
ax.set_xlabel('Stimuli')
ax.set_ylabel('D^2 Performance')
    ax.set_ylim(0,1)
        alphas = np.logspace(-4, -0.5, 20)
        gscv = GridSearchCV(reg, cv=kf, n_jobs=1)
        gscv.fit(i,y)
        # get cv data
        cv_df = pd.DataFrame(gscv.cv_results_)
        ax.semilogx(alphas, scores, c = stim_colors[stim_counter])
        plot error lines showing +/- std. errors of the scores
        scores_std = cv_df.std_test_score
        std_error = scores_std / np.sqrt(5)
        ax.semilogx(alphas, scores + std_error, stim_colors[stim_counter]+'--')
        ax.semilogx(alphas, scores - std_error, stim_colors[stim_counter]+'--')
        ax.fill_between(alphas, scores + std_error, scores - std_error, alpha=0.2)

In [ ]:
# Let's visualize a small part of the design matrix just to see it
# plt.imshow(Xdsgnstim[:200], aspect='auto', interpolation='nearest')
# plt.xlabel('lags before spike time')
# plt.ylabel('time bin of response')
# plt.colorbar()
# plt.show()
# plt.imshow(Xdsgnpupil[:200], aspect='auto', interpolation='nearest')
# Notice it has a structure where every row is a shifted copy of the row
# above, which comes from the fact that for each time bin of response,
# we're grabbing the preceding 'nkt' bins of stimulus as predictor


## Looking at coefficients and trace predictions

In [13]:
plot = False
color_dict = {}
color_list = ['black', 'grey', 'maroon', 'red', 'tomato', 'orange', 'lightyellow', 'greenyellow', 'seagreen', 'teal', 'skyblue',
             'steelblue', 'blue', 'mediumpurple', 'plum', 'magenta', ]
for index, freq in enumerate(np.unique(Stim)): 
    color_dict[freq] = color_list[index]

coef_list = []
performance_list = []
for cell in responsive_cells[true_resp,:]:
    
    responses = np.reshape(cell.flatten(order = 'F'), (-1,1))
    
    # Construct design matrix 
    Xdsgn_final = np.hstack((Xdsgn,responses))
    Xdsgn_final= Xdsgn_final[~np.isnan(Xdsgn_final).any(axis=1)]
    
    X = Xdsgn_final[:,:-1]
    y = Xdsgn_final[:,-1]
    
    # fit tweedie regressor with poisson (power=1)
    reg = TweedieRegressor(power=1, alpha=0.05, fit_intercept=True, link='log', max_iter=1000)
    reg.fit(X=X, y=y)
    coef_list.append(reg.coef_)
    predictions = reg.predict(X)
    performance_list.append(reg.score(X,y))
    if plot == True:
        plotlen=1000
        fig, ax = plt.subplots(figsize = (40,8))
        ax.plot(y[:plotlen],'grey', label = 'Actual')
        ax.plot(np.arange(0,plotlen), predictions[:plotlen], 'cyan', label='Predicted')

        # Mark stimulus onsets
        for stim_index in np.where(Stim != 0)[0]:
            if stim_index < plotlen:
                ax.axvline(x = stim_index,color= color_dict[Stim[stim_index]], label=str(Stim[stim_index]))

    print('D^2 = ', reg.score(X,y))
    
#     axes[0].plot(reg.coef_[:ntfilt])
#     axes[1].plot(reg.coef_[ntfilt:])

coef_array = np.array(coef_list)
stim_coef = np.array(coef_array[:,:ntfilt]).T
pupil_coef = np.array(coef_array[:,ntfilt:]).T
coef_fig, axes = plt.subplots(2,1)
coef_fig.subplots_adjust(hspace=0.7)
axes[0].set_title('Stimulus')
axes[0].set_xlabel('Stimulus Time lag')
axes[0].set_ylabel('Weights');
axes[1].set_title('Pupil')
axes[1].set_xlabel('Pupil Time lag')
axes[1].set_ylabel('Weights');
axes[0].plot(stim_coef);
axes[1].plot(np.mean(pupil_coef));